In [1]:
%config IPCompleter.greedy=True

In [14]:
import os
import rasterio as rio
import in_place
import datetime
import shutil
from rasterio.windows import Window
from matplotlib import pyplot

In [3]:
def get_date_from_filename(filename):
    date = datetime.datetime.strptime(filename[-15:-4], '%d-%b-%Y')
    return date

In [26]:
drive_folder_location = 'C:/Users/karan/Google Drive'
drive_folder = 'test27aug1'
satellite_choice = 'ls8'
block_size = 16 #automate later
number_of_bands = 7
resolution = 30
prediction_dates = ['2014-08-17']
prediction_dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in prediction_dates]

tif_paths = {
    'fine_res_path' : f'{drive_folder_location}/{drive_folder}_{satellite_choice}',
    'mask_path' : f'{drive_folder_location}/{drive_folder}_{satellite_choice}_mask',
    'coarse_res_path' : f'{drive_folder_location}/{drive_folder}_modis'
}
psrfm_info_dst = {
    'PSRFM_exe_path' : 'C:/Users/karan/Documents/CCRS_2A_COOP/PSRFM_Wrapper/PSRM_exe_params',
    'dst_base_path' : 'C:/Users/karan/Documents/CCRS_2A_COOP/PSRFM_Testing/AustraliaAutoTest1',
}
psrfm_info_dst['dst_input_path'] = psrfm_info_dst['dst_base_path'] + '/input'
psrfm_info_dst['dst_param_path'] = psrfm_info_dst['dst_base_path'] + '/params'
psrfm_info_dst['dst_output_path'] = psrfm_info_dst['dst_base_path'] + '/output'
psrfm_info_dst['dst_temp_path'] = psrfm_info_dst['dst_base_path'] + '/temp'



filenames = []
tile_struct = {
    'x_tiles' : 2,
    'y_tiles' : 2}

In [27]:
if not os.path.exists(f'{drive_folder_location}/{drive_folder}'):
    os.mkdir(f'{drive_folder_location}/{drive_folder}')
for path in tif_paths:
#     create destination directories
    if not os.path.exists(psrfm_info_dst['dst_base_path']):
        os.mkdir(psrfm_info_dst['dst_base_path'])
    if not os.path.exists(psrfm_info_dst['dst_input_path']):
        os.mkdir(psrfm_info_dst['dst_input_path'])
    for filename in os.listdir(tif_paths[path]):
        filenames.append(filename)
        with rio.open(f'{tif_paths[path]}/{filename}') as image_to_crop:
#             print(f'[{image_to_crop.meta["width"]}, {image_to_crop.meta["height"]}]')
            finalx = image_to_crop.meta["width"] - image_to_crop.meta["width"] % block_size
            finaly = image_to_crop.meta["height"] - image_to_crop.meta["height"] % block_size
            col_offset = (image_to_crop.width - finalx)/2
            row_offset = (image_to_crop.height - finaly)/2
            
            subset_window = Window(col_offset, row_offset, finalx, finaly)
            newargs = image_to_crop.meta.copy()
            newargs.update({
                'height': subset_window.height,
                'width': subset_window.width,
                'transform': rio.windows.transform(subset_window, image_to_crop.transform),
                'driver': 'ENVI'
            })
            
            with rio.open(f'{psrfm_info_dst["dst_input_path"]}/{filename[:-4]}.dat', 'w', **newargs) as dst:
                dst.write(image_to_crop.read(window=subset_window))

In [28]:
# setting up necessary folders and param files for PSRFM processing
if not os.path.exists(f'{psrfm_info_dst["dst_base_path"]}/PSRFM_Main.exe'):
    shutil.copy(f"{psrfm_info_dst['PSRFM_exe_path']}/PSRFM_Main.exe", psrfm_info_dst['dst_base_path'])
if not os.path.exists(psrfm_info_dst['dst_param_path']):
    os.mkdir(f'{psrfm_info_dst["dst_param_path"]}')
if not os.path.exists(psrfm_info_dst['dst_output_path']):
    os.mkdir(f'{psrfm_info_dst["dst_output_path"]}')

In [34]:
# finding the correct images to put into the param files
for col in range(tile_struct['x_tiles']):
    for row in range(tile_struct['y_tiles']):
        fine_res_filenames = [(filename, get_date_from_filename(filename)) for filename in os.listdir(psrfm_info_dst['dst_input_path']) if filename.startswith(f'{col}_{row}_{satellite_choice}') and not filename.startswith(f'{col}_{row}_{satellite_choice}_mask') and filename.endswith('.dat')]
        mask_filenames = [(filename, get_date_from_filename(filename)) for filename in os.listdir(psrfm_info_dst['dst_input_path']) if filename.startswith(f'{col}_{row}_{satellite_choice}_mask') and filename.endswith('.dat')]
        coarse_res_filenames = [(filename, get_date_from_filename(filename)) for filename in os.listdir(psrfm_info_dst['dst_input_path']) if filename.startswith(f'{col}_{row}_modis') and filename.endswith('.dat')]
        coarse_res_filenames = sorted(coarse_res_filenames, key = lambda x: x[1])
#         find indexes for MODIS images enclosing one set of prediction images
        matching_date_indexes = []
        fine_res_dates = sorted([pair[1] for pair in fine_res_filenames])
        for date in fine_res_dates:
            indx = [coarse_res_filenames.index(tupl) for tupl in coarse_res_filenames if tupl[1] == date]
            matching_date_indexes.append(indx[0])
#     find the filenames of all the files required to run PSRFM and input them into the txt file
        PSRFM_sets = []          
        for index in range(0, len(matching_date_indexes), 2):
            coarse_res_set = coarse_res_filenames[matching_date_indexes[index]:matching_date_indexes[index + 1] + 1]
            fine_res_set = [filename for filename in fine_res_filenames if filename[1] == coarse_res_set[0][1] or filename[1] == coarse_res_set[-1][1]]
            mask_set = [filename for filename in mask_filenames if filename[1] == coarse_res_set[0][1] or filename[1] == coarse_res_set[-1][1]]
            PSRFM_sets.append({
                'coarse_res_filenames' : coarse_res_set,
                'fine_res_filenames': fine_res_set,
                'mask_filenames' : mask_set
            })
#     find how many times PSRFM has to be run and create param files for them
        PSRFM_instances = round(len(matching_date_indexes)/2)
        for instance in range(PSRFM_instances):
            if not os.path.exists(f'{psrfm_info_dst["dst_param_path"]}/{col}_{row}_params_{instance + 1}.txt'):
                shutil.copy(f'{psrfm_info_dst["PSRFM_exe_path"]}/psrfm_template.txt', f'{psrfm_info_dst["dst_param_path"]}/{col}_{row}_params_{instance + 1}.txt')
#             now that file exists, insert the parameters

            with rio.open(f"{psrfm_info_dst['dst_input_path']}/{PSRFM_sets[instance-1]['fine_res_filenames'][0][0]}") as img:
                nrows = img.meta['height']
                ncols = img.meta['width']
                nbands = img.meta['count']
            with fileinput.FileInput(f'{psrfm_info_dst["dst_param_path"]}/{col}_{row}_params_{instance + 1}.txt', inplace = True) as paramfile:
                for line in paramfile:
                    if line.strip().startswith('IN_PAIR_COARSE_FNAME'):
                        print(f"IN_PAIR_COARSE_FNAME = {psrfm_info_dst['dst_input_path']}/{PSRFM_sets[instance-1]['coarse_res_filenames'][0][0]} {psrfm_info_dst['dst_input_path']}/{PSRFM_sets[instance-1]['coarse_res_filenames'][-1][0]}")
                    elif line.strip().startswith('IN_PAIR_FINE_FNAME'):
                        print(f"IN_PAIR_FINE_FNAME = {psrfm_info_dst['dst_input_path']}/{PSRFM_sets[instance-1]['fine_res_filenames'][0][0]} {psrfm_info_dst['dst_input_path']}/{PSRFM_sets[instance-1]['fine_res_filenames'][1][0]}")                                            
                    elif line.strip().startswith('IN_PAIR_FINE_MASK_FNAME'):
                        print(f"IN_PAIR_FINE_MASK_FNAME = {psrfm_info_dst['dst_input_path']}/{PSRFM_sets[instance-1]['mask_filenames'][0][0]} {psrfm_info_dst['dst_input_path']}/{PSRFM_sets[instance-1]['mask_filenames'][1][0]}")                                                     
                    elif line.strip().startswith('IN_PDAY_COARSE_NO'):
                        prediction_filenames = ''
                        for filename in PSRFM_sets[instance-1]['coarse_res_filenames'][1:-1]:
                            prediction_filenames += f" {psrfm_info_dst['dst_input_path']}/{filename[0]}"
                        print(f"IN_PDAY_COARSE_NO = {len(PSRFM_sets[instance-1]['coarse_res_filenames']) - 2} \n {prediction_filenames}")
                    elif line.strip().startswith('OUT_PREDICTION_DIR'):
                        print(f"OUT_PREDICTION_DIR = {psrfm_info_dst['dst_output_path']}")
                    elif line.strip().startswith('OUT_TEMP_DIR'):
                        print(f"OUT_TEMP_DIR = {psrfm_info_dst['dst_temp_path']}")
                    elif line.strip().startswith('OUT_ENVI_HDR'):
                        print(f"OUT_ENVI_HDR = {psrfm_info_dst['dst_input_path']}/{PSRFM_sets[instance-1]['fine_res_filenames'][0][0][:-4]}.hdr")
                    elif line.strip().startswith('NROWS'):
                        print(f"NROWS = {nrows}")
                    elif line.strip().startswith('COARSE_ROWS'):
                        print(f"COARSE_ROWS = {nrows}")
                    elif line.strip().startswith('NCOLS'):
                        print(f"NCOLS = {ncols}")
                    elif line.strip().startswith('COARSE_COLS'):
                        print(f"COARSE_COLS = {ncols}")
                    elif line.strip().startswith('NBANDS'):
                        print(f"NBANDS = {nbands}")
                    elif line.strip().startswith('RESOLUTION'):
                        print(f"RESOLUTION = {resolution}")
                    elif line.strip().startswith('BLOCK_SIZE'):
                        print(f"BLOCK_SIZE = {block_size}")
                    else:
                        print(line, end = '')
# fine_res_filenames, mask_filenames, coarse_res_filenames, PSRFM_instances
# coarse_res_filenames, matching_date_indexes, PSRFM_sets, len(matching_date_indexes)